In [27]:
import os
from pycoingecko import CoinGeckoAPI
import requests

os.environ["OPENAI_API_KEY"] = ''
from openai import OpenAI

client = OpenAI()

# TP 2 - Parte B: Chatbots y Agentes

## Crear nuestro propio chatbot

En la "Parte A" vimos como hacer una llamada a modelos conversacionales (que responden a nuestras instrucciones, como ChatGPT). Vimos que pueden ser usados para resolver varios problemas de NLP.

Y vimos como crear aplicaciones útiles, cómo el whatsappanalyzer. Muchas veces requiriendo no una, sino muchas llamadas al modelo y combinándolas: usando el texto de alguna llamada anterior para construir el prompt.

**Nota:** notar que por prompt nos referimos a toda la conversación que le pasamos a la API call en el campo messages.

Crear un chatbot no es más que una aplicación más que podemos hacer, pero que merece atención especial ya que es una aplicación frecuentemente requerida.

A continaución está el código de un chatbot que programé haciendo llamadas a las API de ChatGPT.

#### Ejercicio B.1

A continuación hay un chatbot que he programado pero tiene un error. Debido al error el chatbot no tiene memoria: no recuerda qué se conversó en mensajes anteriores. Modificalo de modo que tenga memoria.
Una forma de corroborar que la memoria está correctamente implementada es darle un dato fáctico y preguntarle por el mismo a continuación.Por ejemplo probá decirle tu nombre y preguntale en un mensaje sucesivo cómo te llamás. 

In [6]:
print("Welcome to the OpenAI Chat! Type 'exit' to end the chat.\n")

while True:
        history = []
        user_input = input("User: ")
        if user_input.lower().strip() == "exit":
            break
            
        history.append({"role": "user", "content": user_input})

        response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=history
        )

        assistant_message = response.choices[0].message.content.strip()
        print(f"Assistant: {assistant_message}\n")
        history.append({"role": "assistant", "content": assistant_message})

print("Chat ended.")

Welcome to the OpenAI Chat! Type 'exit' to end the chat.



User:  hola me llamo mariano


Assistant: ¡Hola Mariano! ¿En qué puedo ayudarte hoy?



User:  como me llamo?


Assistant: No tengo la capacidad de saber tu nombre, ya que soy un asistente virtual y no tengo acceso a esa información. ¿En qué más puedo ayudarte?



User:  exit


Chat ended.


## Agentes

Los agentes en IA son un concepto bastante viejo, del siglo pasado. Muy anterior a la existencia de LLMs.

Conceptualmente, son sistemas que interactúan con el ambiente, produciendo acciones en el mismo.

![alt text](agents-in-ai.png "Agent")

El concepto central del machine learning actual es el de modelo. El modelo es un sistema que hace predicciones. Notar que si bien un modelo puede ser parte de un agente son conceptualmente distintos. Mientras que el modelo quiere predecir algo, el agente tiene además la capacidad de actuar: la relación entre la predicción del modelo y la acción llevada a cabo por el agente no es trivial y tiene que ser estudiada con cuidado.

#### Prediciendo bien, actuando mal

Como anécdota ilustrativa para marcar esta diferencia conceptual está el caso de una startup de real estate que entrenó un modelo para predecir el precio de propiedades. 

Si bien conocían el márgen de error promedio, luego cuando lo usaron para tomar decisión de qué inmuebles comprar y a qué precio terminaron teniendo grandes pérdidas.

Lo primero fue pensar que tenían un error en la predicción o que el error era más grande del que habían calculado.

El análisis post-mortem arrojó que el problema había sido otro: ellos ofertaban un precio de compra que, en promedio, tenía un error como el calculado. Los vendedores, que poseían más información acerca de sus inmuebles, aceptaban esta oferta cuando el precio les era conveniente y la rechazaban cuando no. O sea, que si bien el error era promedio las compras sólo se efectuaban en casos donde la casa había sido tasada en ventaja para el vendedor, generando una desventaja sistemática para la compañía.

## Agentes y LLMs

Vimos que los modelos conversacionales responden a nuestras instrucciones con precisión. Entonces pueden ser usados como el "corazón" de los agentes: le podemos dar instrucciones para que, dado un escenario y un conjunto de acciones decidan qué acción aplicar. Estas acciones repercutirán en el escenario, y luego le presentaremos un nuevo escenario y otro conjuntos de acciones.

Es un concepto bastante amplio. así que lo mejor es verlo con un ejemplo. 
Ahora imaginemos que queremos agregarle a nuestro chat anterior la capacidad de contestar el clima que hace en cierta ciudad.
Entonces crearemos un sistema que decida que acción realizar:

- Responderle al usuario con cierto texto.
- Consultar al servicio meteorológico de cierta ciudad.

A veces la respuesta de los LLMs no es robusta: le pedimos que responda con un JSON y nos da una string que no parsea como JSON o le pedimos que ciertos campos estén presentes y no lo están. El código que maneje la respuesta, la parsee y la traduzca en acciones debiera manejar este tipo de errores para que la aplicación no crashee cuando sucedan. E

vitarlos a nivel LLM es un área de investigación activa y un problema no resuelto, en nuestro agente (más abajo) incluímos como último mensaje la consigna que dice como debiera ser el formato de la respuesta (por más que también fue incluído como primer mensaje). Algunos resultados prematuros arrojan que los LLM prestan más atención al último mensaje y en conversaciones largas "olvidan" los primeros.

#### Ejercicio B.2

Entender el código del chat, leer la conversación ejemplo que se incluye en esta notebook.
Agregar una ciudad nueva y usarlo para preguntarle el clima en esa ciudad.

In [23]:
print("Welcome to the OpenAI Chat! Type 'exit' to end the chat.\n")
import json
instruction_prompt = """
REMINDER ABOUT THE FORMAT OF THE ANSWER:
You are an agent that can decide to:
- reply to the user.
- query the weather service.

You are going to be provided with the message history with an user. As a result you must produce a JSON with the fields:
- response_type (manadatory field): 'reply_to_user' | 'query_weather' if you want to reply to the user or query the weather service.
- city (only if response_type is 'query_weather'): str the name of the city to ask to the weather service.
- response (only if response_type is 'reply_to_user'): str the response to the user.
""".strip()
weather_service_api = {'rosario': 'cloudy', 'santa fe': 'sunny'}
history = [{"role": "system", "content": instruction_prompt}]
skip_input = False
while True:
    if not skip_input:
        user_input = input("User: ")
        if user_input.lower().strip() == "exit":
            break

        history.append({"role": "user", "content": user_input})
    skip_input = False

    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=history+[{"role": "user", "content": instruction_prompt}]
    )

    agent_response_raw = response.choices[0].message.content.strip()
    #print(agent_response_raw)
    agent_response_json = json.loads(agent_response_raw)
    if agent_response_json['response_type'] == 'reply_to_user':
        print(f"Assistant: {agent_response_json['response']}\n")
        history.append({"role": "assistant", "content": agent_response_json['response']})
    elif agent_response_json['response_type'] == 'query_weather':
        city = agent_response_json['city'].lower().strip()
        if city in weather_service_api:
            weather = weather_service_api[city]
            extra_info_added = f"(The system weather shows that the weather for the cit {city} is {weather})"
        else:
            extra_info_added = f"(The system weather shows that the weather for the cit {city} is unavailable, please do not ask again for this city)"
        history.append({"role": "user", "content": extra_info_added})
        skip_input = True
        

print("Chat ended.")

Welcome to the OpenAI Chat! Type 'exit' to end the chat.



User:  Hola como estas?


Assistant: Hola! Estoy aquí para ayudarte. ¿En qué puedo asistirte hoy?



User:  Bien, mi nombre es Mariano, estas listo para las preguntas?


Assistant: ¡Hola Mariano! Estoy listo para responder tus preguntas. Adelante, ¿en qué puedo ayudarte?



...

Assistant: Es posible que haya un problema temporal con la información del clima para Buenos Aires. Te recomendaría intentarlo más tarde o consultar otro servicio meteorológico. ¿Puedo ayudarte con algo más?



User:  Y en la capital de la provincia donde rosario esta ubicada?


Assistant: La capital de la provincia donde se encuentra Rosario, Santa Fe, tiene un clima soleado en este momento.



User:  exit


Chat ended.


#### Ejercicio B.3

Hacer un caso de uso de agente siguiendo el ejemplo anterior como guía.

## Herramientas (tools)

Estas acciones desencadenan, como en ejemplo anterior llamadas a servicios externos (como weather_service_api, como podría ser consultas a base de datos, a una calculadora, etc), es por ello que se llaman herramientas. Ya que permiten al modelo usar cierto servicio externo como herramienta.

Una manera de que el modelo entienda las acciones que tiene disponibles y el formato para llamarlas es incluir casos de uso de acciones durante el entrenamiento. Los proveedores de servicios como chatgtp permiten conectar los modelos con herramientas, incluyendo en su API un campo para especificarle el formato con el que el modelo debe llamar a la herramienta (sus campos y la semántica de los campos y resultados). Notar que igual que pasaba en el ejemplo anterior: el que hace la llamda al modelo (o sea, nosotros) es el responsable de implementar las llamadas a las herramientas. El modelo solo dirá "quiero llamar a la herramienta X con los argumentos A, B, C".

Las API también incluyen una manera de incluir respuestas a llamadas a herramientas del modelo. De modo que si el modelo nos contesta diciendo que necesita usar get_weather_service para la ciudad 'rosario', el siguiente request además de todo el historial le contestará "la respuesta a la función get_weather-service que hiciste en tu solicitud anterior es ...".

A continuación implementamos dos funciones que luego le daremos a nuestro asistente:
- Una que permite conocer las condiciones del clima en cierta ciudad.
- Otra que permite buscar datos en tiempo real de cotizaciones de criptomonedas.

** Llamadas a funciones**     

Las llamadas a funciones de GPT permiten 

**Ejercicio opcional:** Siguiendo el ejemplo brindado, crear tu propias funciones para potenciar al asistente.

**Ejemplo:** Escribir funciones para:
- Permitirle al asistente obtener datos en tiempo real de las condiciones climatológicas en una ubicación determinada. El Asistente deberá poder responder acertadamente a consultas como "¿Cuál es actualmente la temperatura en México DF? Detallar otras condiciones climatológicas de ser posible"
- Permitirle al asistente buscar datos en tiempo real de cotizaciones de criptomonedas.

In [41]:
def get_weather(city):
    api_key = '7663c30fc4c275ea5f373e7052d8a3d3'
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric&lang=es'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'description': data['weather'][0]['main'],
            'humidity': data['main']['humidity'],
            'pressure': data['main']['pressure']
        }
        return weather
    else:
        return {'error': 'Weather data could not be found for the specified city.'}

get_weather('Mexico')

{'city': 'Mexico',
 'temperature': 25.84,
 'description': 'Clouds',
 'humidity': 96,
 'pressure': 1010}

In [42]:
def get_crypto_price(currency):
    cg = CoinGeckoAPI()
    data = cg.get_price(ids=currency.lower(), vs_currencies='usd')
    if data:
        price = data.get(currency.lower(), {}).get('usd')
        if price:
            return {'currency': currency.capitalize(), 'price_usd': price}
    return {'error': 'Cound not find price for the specified cryptocurrency.'}

get_crypto_price('bitcoin')

{'currency': 'Bitcoin', 'price_usd': 76401}

In [43]:
debug_mode = False

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Gets the current weather of a specific city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "Name of the city, for example, 'Mexico City'."
                    },
                },
                "required": ["city"],
                "additionalProperties": False,
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_crypto_price",
            "description": "Gets the current price in USD of a specific cryptocurrency.",
            "parameters": {
                "type": "object",
                "properties": {
                    "currency": {
                        "type": "string",
                        "description": "Name of the cryptocurrency, for example, 'bitcoin', 'ethereum'."
                    }
                },
                "required": ["currency"],
                "additionalProperties": False,
            },
        }
    }
]

messages = []

print("Welcome to the OpenAI Chat with Function Calling! Type 'exit' to end the chat.\n")

while True:
    try:
        user_input = input("User: ")

        if user_input.lower() == 'exit':
            print("Chat ended.")
            break

        messages.append({"role": "user", "content": user_input})

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # Use a model that supports function calling
            messages=messages,
            tools=tools,
        )

        # Extract the assistant's message
        assistant_message = response.choices[0].message

        if debug_mode:
            print(assistant_message)

        # Check if the assistant wants to call a function
        if assistant_message.tool_calls:
            messages.append(assistant_message)  # Assistant's function call
            
            for tool_call in assistant_message.tool_calls:
                function_args = json.loads(tool_call.function.arguments)
                function_name = tool_call.function.name
    
                if debug_mode:
                    print(f"\nAssistant is calling function: {function_name} with arguments {function_args}\n")
    
                # Call the appropriate function based on the function name
                if function_name == "get_weather":
                    function_response = get_weather(**function_args)
                elif function_name == "get_crypto_price":
                    function_response = get_crypto_price(**function_args)
                else:
                    function_response = {"error": "Function not recognized."}
    
                # Convert the function response to a JSON-formatted string
                function_response_json = json.dumps(function_response)

                function_call_result_message = {
                    "role": "tool",
                    "content": function_response_json,
                    "tool_call_id": tool_call.id
                }
                messages.append(function_call_result_message)

            # Get the assistant's final response using the function's output
            second_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages
            )

            # Extract and print the assistant's final answer
            final_answer = second_response.choices[0].message.content
            print(f"Assistant: {final_answer}\n")

            # Append the final answer to the conversation
            messages.append({"role": "assistant", "content": final_answer})
        else:
            # If no function call, just print the assistant's response
            assistant_content = assistant_message.content.strip()
            print(f"Assistant: {assistant_content}\n")
            messages.append(assistant_message)

    except Exception as e:
        print(f"An error occurred: {e}")
        break


Welcome to the OpenAI Chat with Function Calling! Type 'exit' to end the chat.



User:  which is the wather of mexico df and price of bitcoin


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Zyq6yAjbIwozSoNdxwwf03vk', function=Function(arguments='{"city": "Mexico City"}', name='get_weather'), type='function'), ChatCompletionMessageToolCall(id='call_t2TXIFsrFh50YFAiLrt303b5', function=Function(arguments='{"currency": "bitcoin"}', name='get_crypto_price'), type='function')], refusal=None)

Assistant is calling function: get_weather with arguments {'city': 'Mexico City'}


Assistant is calling function: get_crypto_price with arguments {'currency': 'bitcoin'}

Assistant: The weather in Mexico City is currently cloudy with a temperature of 24.97°C, humidity of 30%, and a pressure of 1013 hPa. 

The price of Bitcoin is $76,401 USD.



User:  What's the last message?


ChatCompletionMessage(content='The last message was:\n"The price of Bitcoin is $76,401 USD."', role='assistant', function_call=None, tool_calls=None, refusal=None)
Assistant: The last message was:
"The price of Bitcoin is $76,401 USD."



User:  exit


Chat ended.


#### Ejercicio B.4

Implementá tu propio agente. Notá que las tools no necesariamente tienen que llamar a la API. Pueden:
- Hacer una cuenta matemática que vos conozcas.
- Ejecutar un algoritmo para modificar el entorno.
- Las funciones disponibles pueden depender del entorno (dejar de estar disponibles / pasara estar disponibles).

Por ejemplo, podríamos crear un laberinto con puertas y llaves y el modelo podría tener como acciones dar un paso en ese laberinto en alguna dirección, recoger una llamer, usar una llave que tenga en el inventario, etc. Definitivamente la idea sería muy distinto al ejemplo de arriba pero el concepto es el mismo: llamar repetidas veces a una LLM y usar la respuesta del mismo para modificar el entorno y consecuentemente la siguiente llamada.

No te limites: el corazón de esto es hacer varias llamadas a LLMs y usar la respuesta del mismo para modificar el entorno y consecuentemente la siguiente llamada. No es necesario que el agente que hagas funcione o que sea útil. La capacidad de razocinio de los LLM es sorprendente pero a veces pueden ser sorprendentemente estúpidos y todavía conocer sus limitaciones es un área de investigación activa donde hay pocas luces.